In [1]:
import os
import copy
import gc

import numpy as np
import pandas as pd

# Data Analysis

In [2]:
file_path = "/Users/rongzhimai/downloads/Brownie.csv"

df = pd.read_csv(file_path)
df.head()

,Name,Rating,Rarity,Preferred Position,Nation,League,Team,Price Limits,Last Sale Price,Discard Value,Contract,Untradeable,Loans,DefinitionId,IsDuplicate,IsInActive11,Alternate Positions,ExternalPrice
0,Lewandowski,94,UCL Road to the Knockouts,ST,Poland,LALIGA EA SPORTS,FC Barcelona,Min:60000 Max:1100000,0,0,16,True,False,67297409,False,True,"CF,ST",558000
1,Maldini,93,Special,CB,Italy,Icons,ICON,Min:400000 Max:7600000,0,0,2,True,True,50570087,False,True,"CB,LB",4250000
2,Mbappé,92,Special,ST,France,Ligue 1 Uber Eats,Paris SG,Min:2000 Max:20000,0,0,6,True,False,84117827,False,False,"CF,ST,LW",3718750
3,Henry,91,Special,ST,France,Icons,ICON,Min:170000 Max:3200000,0,0,11,True,True,1625,False,False,"CF,ST,LW",1929000
4,Haaland,91,Rare,ST,Norway,Premier League,Manchester City,Min:10500 Max:110000,0,0,19,True,True,239085,False,False,"CF,ST",70000


In [156]:
def generate_quality(rating):
    if rating >= 81:
        return "dont touch"
    
    if rating >= 75:
        return "gold"
    
    if rating >= 65:
        return "silver"
    
    return "bronze"

In [157]:
def get_quality_combo(data, quality, target):
    return data.loc[data["quality"].isin(quality), target].value_counts()

In [158]:
df["quality"] = df["Rating"].apply(generate_quality)

In [159]:
df.quality.value_counts()

quality
dont touch    266
silver        119
gold          104
bronze         45
Name: count, dtype: int64

In [160]:
df.League.unique()

array(['LALIGA EA SPORTS', 'Bundesliga', 'Icons', 'Premier League',
       'D1 Arkema', 'Barclays WSL', 'Liga F', 'GPFBL', 'Serie A TIM',
       'Ligue 1 Uber Eats', 'NWSL', 'Trendyol Süper Lig',
       'Calcio A Femminile', 'ROSHN Saudi League', 'EFL Championship',
       'MLS', 'Eredivisie', 'Allsvenskan', 'Liga Portugal',
       'Libertadores', 'Hellas Liga', 'Serie BKT', 'cinch Prem',
       '1A Pro League', 'Sudamericana', 'Bundesliga 2',
       'LALIGA HYPERMOTION', 'CSSL ', 'Sverige Liga', 'LPF',
       'Magyar Liga', 'Liga Hrvatska', 'CSL', 'Ligue 2 BKT',
       '3F Superliga', 'EFL League One', 'PKO BP Ekstraklasa', 'A-League',
       'Ö. Bundesliga', 'SUPERLIGA', 'Eliteserien', 'EFL League Two',
       '3. Liga'], dtype=object)

In [171]:
df.groupby("Team").size().sort_values().tail(10)

Team
OL                   10
VfL Wolfsburg        11
Manchester Utd       11
Juventus             11
Chelsea              12
Manchester City      12
FC Bayern München    15
Arsenal              15
FC Barcelona         18
Paris SG             20
dtype: int64

In [133]:
df[df["Rating"].eq(84)]

,Name,Rating,Rarity,Preferred Position,Nation,League,Team,Price Limits,Last Sale Price,Discard Value,Contract,Untradeable,Loans,DefinitionId,IsDuplicate,IsInActive11,Alternate Positions,ExternalPrice,quality
149,Rodman,84,Rare,RW,United States,NWSL,Washington Spirit,Min:1100 Max:10000,0,0,4,True,True,266928,False,False,"RM,RW,ST",5700,dont touch
150,Marta,84,Rare,CAM,Brazil,NWSL,Orlando Pride,Min:1100 Max:14000,0,0,5,True,True,226890,False,False,"RM,CAM",4100,dont touch
151,Foord,84,Rare,LW,Australia,Barclays WSL,Arsenal,Min:1100 Max:10000,3200,596,7,False,False,227110,False,False,"LM,CAM,RW,LW",3700,dont touch
152,James,84,Rare,RB,England,Premier League,Chelsea,Min:1100 Max:10000,0,596,7,False,False,238074,False,False,"RWB,RB",3800,dont touch
153,Gabriel Jesus,84,Rare,ST,Brazil,Premier League,Arsenal,Min:1100 Max:10000,0,596,7,False,False,230666,False,False,"CF,RW,ST",4500,dont touch
154,Lukaku,84,Rare,ST,Belgium,Serie A TIM,Roma FC,Min:1100 Max:10000,3100,596,4,False,False,50524153,False,False,"CF,ST",3700,dont touch
155,Bremer,84,Rare,CB,Brazil,Serie A TIM,Juventus,Min:1100 Max:10000,3200,596,4,False,False,239580,False,False,CB,4200,dont touch
156,De Paul,84,Rare,CM,Argentina,LALIGA EA SPORTS,Atlético de Madrid,Min:1100 Max:10000,0,596,7,False,False,212616,False,False,CM,3900,dont touch
157,Nagasato,84,Rare,CAM,Japan,NWSL,Chicago Red Stars,Min:1100 Max:10000,0,596,7,False,False,267275,False,False,"CM,CAM,ST",4000,dont touch
158,Raphinha,84,Rare,RW,Brazil,LALIGA EA SPORTS,FC Barcelona,Min:1100 Max:10000,0,0,28,True,False,233419,False,False,"RM,RW",4100,dont touch


In [118]:
pd.concat(
    [
        df[df["Untradeable"].eq(True) & df["Rating"].ge(83)],
        df[df["Rating"].lt(83)]
    ]
).groupby("Rating", as_index=False).size()

,Rating,size
0,54,1
1,57,1
2,58,1
3,60,3
4,61,3
5,62,6
6,63,5
7,64,14
8,65,6
9,66,10


In [26]:
get_quality_combo(df, ["gold", "bronze", "silver"], "Nation").head(30)

France           22
Spain            21
Argentina        21
England          17
Germany          16
United States    14
Italy            12
Brazil           11
Turkey           11
Portugal         10
Netherlands       9
Uruguay           9
Denmark           8
Colombia          6
Belgium           6
Sweden            5
Japan             5
Ecuador           5
Romania           4
Venezuela         4
Scotland          4
Poland            4
Switzerland       4
Austria           4
Morocco           4
Paraguay          3
Ghana             3
Wales             3
Chile             3
Canada            3
Name: Nation, dtype: int64

In [111]:
os.remove(file_path)

# Mbappe Squads

In [2]:
sequence = [
    [0,0,0,6,0,1,4,0,0],
    [0,0,1,4,1,1,4,0,0],
    [0,0,0,6,0,3,0,2,0],
    [0,0,2,5,0,0,2,2,0],
    [0,0,2,5,0,0,2,2,0],
    [4,0,0,0,0,2,5,0,0],
    [0,0,0,3,3,2,3,0,0],
    [3,2,0,0,0,0,5,1,0],
    [0,0,0,1,6,2,1,1,0],
    [0,0,0,1,6,1,3,0,0],
    [0,0,0,1,3,6,1,0,0],
    [0,0,0,2,6,1,0,2,0],
    [0,0,0,0,6,3,2,0,0],
    [0,0,0,3,3,4,0,0,1],
    [0,0,0,0,6,4,0,1,0],
    [0,4,1,0,1,0,4,0,1],
    [0,0,0,0,9,0,0,2,0],
    [0,0,0,1,6,3,0,0,1],
    [0,0,0,2,7,0,0,1,1],
    [1,0,0,0,0,10,0,0,0],
    [0,0,1,0,8,0,0,1,1],
    [0,0,0,0,2,9,0,0,0]
]

In [3]:
comb = {}

for i in range(82, 83+1):
    comb[i] = []
    
    for idx1, s in enumerate(sequence):
        comb[i].append({})
        
        for idx2, j in enumerate(range(i-5, i+4)):
            comb[i][-1][f"p{j}"] = s[idx2]
        
        comb[i][-1]["p00"] = idx1 + 1

In [133]:
def get_price(row, prices):
    ret = 0
    
    for i in range(80, 92+1):
        if f"p{i}" not in row:
            continue
        
        ret += prices[i] * row[f"p{i}"]
    
    return ret

In [113]:
tradeable = {
    80: 0,
    81: 0,
    82: 0,
    83: 0,
    84: 30,
    85: 12,
    86: 17,
    87: 6,
    88: 11,
    89: 3,
    90: 0,
    91: 1,
    92: 0
}

untradeable = {
    80: 99,
    81: 99,
    82: 99,
    83: 99,
    84: 32,
    85: 13,
    86: 4,
    87: 4,
    88: 4,
    89: 3,
    90: 1,
    91: 3,
    92: 2
}

transfer_list = {
    80: 0,
    81: 0,
    82: 0,
    83: 0,
    84: 23,
    85: 4,
    86: 6,
    87: 10,
    88: 36,
    89: 3,
    90: 0,
    91: 0,
    92: 0
}

prices = {
    80: 650,
    81: 650,
    82: 750,
    83: 1000,
    84: 3100,
    85: 6900,
    86: 12000,
    87: 16000,
    88: 25000,
    89: 36000,
    90: 0,
    91: 0,
    92: 0
}

In [122]:
scaled_prices = {}
limits = {}

for i in range(80, 92+1):
    scaled_prices[i] = prices[i] * (tradeable[i] + transfer_list[i]) / (tradeable[i] + transfer_list[i] + untradeable[i])
    limits[i] = tradeable[i] + transfer_list[i] + untradeable[i]

In [123]:
scaled_prices

{80: 0.0,
 81: 0.0,
 82: 0.0,
 83: 0.0,
 84: 1932.9411764705883,
 85: 3806.896551724138,
 86: 10222.222222222223,
 87: 12800.0,
 88: 23039.21568627451,
 89: 24000.0,
 90: 0.0,
 91: 0.0,
 92: 0.0}

In [124]:
limits

{80: 99,
 81: 99,
 82: 99,
 83: 99,
 84: 85,
 85: 29,
 86: 27,
 87: 20,
 88: 51,
 89: 9,
 90: 1,
 91: 4,
 92: 2}

In [7]:
def backtrack(_map, tmp, base, i, depth, limits, use_limits=True):
    print(f"@@@ depth = {i+1} @@@")
    if i == depth - 1:
        for t in tmp:
            counter = 0
            
            if use_limits:
                for k, v in t.items():
                    if k == "p00":
                        continue
                    
                    if v >= limits[int(k[1:])]:
                        counter += 1
                        break
            
            if counter == 0:    
                _map.append(copy.deepcopy(t))
                
        return
    
    curr_tmp = []
    for t in tmp:
        for b in base:
            curr_tmp.append({})
            
            t_copy = copy.deepcopy(t)
            b_copy = copy.deepcopy(b)
            
            for k, v in t_copy.items():
                if k == "p00":
                    continue
                
                if k not in curr_tmp[-1]:
                    curr_tmp[-1][k] = 0
                
                curr_tmp[-1][k] += (v + b_copy[k])
            
            curr_tmp[-1]["p00"] = f"{t_copy['p00']}_{b_copy['p00']}"
            
    curr_df = pd.DataFrame(curr_tmp)
    cols = list(curr_df.columns)
    cols.remove("p00")
    
    curr_df.drop_duplicates(subset=cols, ignore_index=True, inplace=True)
    
    new_tmp = curr_df.to_dict("records")
            
    # print(curr_tmp)
    backtrack(_map, new_tmp, base, i+1, depth, limits, use_limits)

In [5]:
limits = {i: 99 for i in range(77, 88)}
limits

{77: 99,
 78: 99,
 79: 99,
 80: 99,
 81: 99,
 82: 99,
 83: 99,
 84: 99,
 85: 99,
 86: 99,
 87: 99}

In [8]:
map82 = []
map83 = []

backtrack(map82, comb[82], comb[82], 0, 1, limits)
backtrack(map83, comb[83], comb[83], 0, 1, limits)

@@@ depth = 1 @@@
@@@ depth = 1 @@@


In [149]:
map85 = []
map87 = []
map88 = []
map89 = []

print("processing 85")
backtrack(map85, comb[85], comb[85], 0, 1, limits)
print("processing 87")
backtrack(map87, comb[87], comb[87], 0, 5, limits)
print("processing 88")
backtrack(map88, comb[88], comb[88], 0, 4, limits)
print("processing 89")
backtrack(map89, comb[89], comb[89], 0, 3, limits)

processing 85
@@@ depth = 1 @@@
processing 87
@@@ depth = 1 @@@
@@@ depth = 2 @@@
@@@ depth = 3 @@@
@@@ depth = 4 @@@
@@@ depth = 5 @@@
processing 88
@@@ depth = 1 @@@
@@@ depth = 2 @@@
@@@ depth = 3 @@@
@@@ depth = 4 @@@
processing 89
@@@ depth = 1 @@@
@@@ depth = 2 @@@
@@@ depth = 3 @@@


In [150]:
df85 = pd.DataFrame(map85)
df87 = pd.DataFrame(map87)
df88 = pd.DataFrame(map88)
df89 = pd.DataFrame(map89)

In [10]:
df82 = pd.DataFrame(map82)
df83 = pd.DataFrame(map83)

In [151]:
df85.shape, df87.shape, df88.shape, df89.shape

((22, 10), (10249, 10), (728, 10), (19, 10))

In [13]:
def cross(ls1, ls2, limits, threshold):
    ret = []
        
    keys_1 = set(ls1[0].keys())
    keys_2 = set(ls2[0].keys())
    
    keys = sorted(keys_1.union(keys_2))
    keys.remove("p00")
    
    for l1 in ls1:
        for l2 in ls2:
            curr_map = {}
            
            for _key in keys:
                v1 = l1.get(_key, 0)
                v2 = l2.get(_key, 0)
                                
                curr_map[_key] = v1 + v2
            
            counter = 0
            for k, v in curr_map.items():
                if v > limits[int(k[1:])]:
                    counter += (v - limits[int(k[1:])])
                    
                if counter > threshold:
                    break
                    
            if counter > threshold:
                continue
            
            p00 = f"{l1['p00']}_{l2['p00']}"
            curr_map["p00"] = p00
            ret.append(copy.deepcopy(curr_map))
        
    df = pd.DataFrame(ret)
    cols = list(df.columns)
    cols.remove("p00")
    
    df.drop_duplicates(subset=cols, ignore_index=True, inplace=True)
    print(df.shape[0])
    
    return df

In [18]:
res = cross(
    df82[df82["p84"]>=2].to_dict("records"), 
    df83[df83["p85"]>=1].to_dict("records"), 
    limits, 
    threshold=0
)

36


In [26]:
df82[df82["p00"].eq(4)]

,p77,p78,p79,p80,p81,p82,p83,p84,p85,p00
3,0,0,2,5,0,0,2,2,0,4


In [24]:
df83[df83["p00"].eq(15)]

,p78,p79,p80,p81,p82,p83,p84,p85,p86,p00
14,0,0,0,0,6,4,0,1,0,15


In [22]:
res[res["p85"].eq(1) & res["p84"].eq(2) & res["p86"].eq(0)]

,p77,p78,p79,p80,p81,p82,p83,p84,p85,p86,p00
5,0,0,0,6,0,9,4,2,1,0,3_15
14,0,0,2,5,0,6,6,2,1,0,4_15
23,0,0,0,2,6,7,4,2,1,0,12_15
32,0,0,0,0,9,6,4,2,1,0,17_15


In [130]:
times = {
    89: 2, 88: 2, 87: 4
}

map89 = []
backtrack(map89, comb[89], comb[89], 0, 1, limits)
ret = pd.DataFrame(map89)

for rating, squads in times.items():
    print(f"running rating = {rating}")
    
    for s in range(squads):
        print(f"@@@ running squad no.{s+1} @@@")
        
        curr_map = []
        backtrack(curr_map, comb[rating], comb[rating], 0, 1, limits)
        curr_df = pd.DataFrame(curr_map)
        
        ret = cross(ret.to_dict("records"), curr_df.to_dict("records"), limits, threshold=0)

@@@ depth = 1 @@@
running rating = 89
@@@ running squad no.1 @@@
@@@ depth = 1 @@@
35
@@@ running squad no.2 @@@
@@@ depth = 1 @@@
50
running rating = 88
@@@ running squad no.1 @@@
@@@ depth = 1 @@@
335
@@@ running squad no.2 @@@
@@@ depth = 1 @@@
936
running rating = 87
@@@ running squad no.1 @@@
@@@ depth = 1 @@@
10057
@@@ running squad no.2 @@@
@@@ depth = 1 @@@
51277
@@@ running squad no.3 @@@
@@@ depth = 1 @@@
164338
@@@ running squad no.4 @@@
@@@ depth = 1 @@@
345043


In [131]:
ret["high_values"] = ret["p88"] + ret["p89"] + ret["p90"] + ret["p91"] + ret["p92"]

In [134]:
ret["price"] = ret.apply(lambda x: get_price(x, prices), axis=1)

In [135]:
ret.describe()

,p82,p83,p84,p85,p86,p87,p88,p89,p90,p91,p92,high_values,price
count,345043.000000,345043.000000,345043.000000,345043.000000,345043.000000,345043.000000,345043.000000,345043.000000,345043.0,345043.000000,345043.000000,345043.000000,3.450430e+05
mean,2.874572,1.702944,1.075165,8.758827,12.305301,14.485171,45.094284,7.258794,0.0,3.659964,1.784978,57.798019,1.835728e+06
std,3.052491,1.692361,1.306401,5.102434,6.038299,3.909208,3.787310,1.738009,0.0,0.556075,0.423763,3.117569,5.089035e+04
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31.000000,0.000000,0.0,2.000000,0.000000,46.000000,1.727600e+06
25%,0.000000,0.000000,0.000000,5.000000,8.000000,12.000000,42.000000,6.000000,0.0,3.000000,2.000000,56.000000,1.792100e+06
50%,3.000000,1.000000,1.000000,8.000000,12.000000,15.000000,45.000000,8.000000,0.0,4.000000,2.000000,58.000000,1.828700e+06
75%,4.000000,3.000000,2.000000,12.000000,16.000000,18.000000,48.000000,9.000000,0.0,4.000000,2.000000,60.000000,1.873400e+06
max,16.000000,11.000000,9.000000,26.000000,27.000000,20.000000,51.000000,9.000000,0.0,4.000000,2.000000,66.000000,2.009000e+06


In [141]:
comb[89][2], comb[89][18], comb[89][18], comb[88][5], comb[88][10], comb[87][0]

({'p84': 0,
  'p85': 0,
  'p86': 0,
  'p87': 6,
  'p88': 0,
  'p89': 3,
  'p90': 0,
  'p91': 2,
  'p92': 0,
  'p00': 3},
 {'p84': 0,
  'p85': 0,
  'p86': 0,
  'p87': 2,
  'p88': 7,
  'p89': 0,
  'p90': 0,
  'p91': 1,
  'p92': 1,
  'p00': 19},
 {'p84': 0,
  'p85': 0,
  'p86': 0,
  'p87': 2,
  'p88': 7,
  'p89': 0,
  'p90': 0,
  'p91': 1,
  'p92': 1,
  'p00': 19},
 {'p83': 4,
  'p84': 0,
  'p85': 0,
  'p86': 0,
  'p87': 0,
  'p88': 2,
  'p89': 5,
  'p90': 0,
  'p91': 0,
  'p00': 6},
 {'p83': 0,
  'p84': 0,
  'p85': 0,
  'p86': 1,
  'p87': 3,
  'p88': 6,
  'p89': 1,
  'p90': 0,
  'p91': 0,
  'p00': 11},
 {'p82': 0,
  'p83': 0,
  'p84': 0,
  'p85': 6,
  'p86': 0,
  'p87': 1,
  'p88': 4,
  'p89': 0,
  'p90': 0,
  'p00': 1})

In [145]:
ret.sort_values("price")

,p82,p83,p84,p85,p86,p87,p88,p89,p90,p91,p92,p00,high_values,price
26493,0,4,0,24,1,17,38,9,0,4,2,3_19_19_6_11_1_1_1_1,53,1727600
26494,0,4,1,22,2,17,38,9,0,4,2,3_19_19_6_11_1_1_1_2,53,1728900
26499,0,4,2,20,3,17,38,9,0,4,2,3_19_19_6_11_1_1_2_2,53,1730200
26495,4,4,0,18,1,18,39,9,0,4,2,3_19_19_6_11_1_1_1_6,54,1730200
26500,4,4,1,16,2,18,39,9,0,4,2,3_19_19_6_11_1_1_2_6,54,1731500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186820,0,0,0,4,24,9,49,9,0,4,0,15_15_17_22_22_9_10_10_10,62,2008600
186821,0,0,0,3,24,11,48,9,0,4,0,15_15_17_22_22_9_10_10_13,61,2008700
186824,0,0,0,2,24,13,47,9,0,4,0,15_15_17_22_22_9_10_13_13,60,2008800
186828,0,0,0,1,24,15,46,9,0,4,0,15_15_17_22_22_9_13_13_13,59,2008900


In [71]:
cross_85_88 = cross(df85.to_dict("records"), df88.to_dict("records"), limits, threshold=0)
cross_85_88.head(10)

14124


,p80,p81,p82,p83,p84,p85,p86,p87,p88,p89,p90,p91,p00
0,0,0,0,6,0,1,22,6,7,12,1,0,1_1_1_1_15
1,0,0,0,7,0,1,22,0,13,12,0,0,1_1_1_1_20
2,0,0,0,6,0,1,22,2,12,12,0,0,1_1_1_1_22
3,0,0,0,6,0,2,20,7,7,12,1,0,1_1_1_2_15
4,0,0,0,7,0,2,20,1,13,12,0,0,1_1_1_2_20
5,0,0,0,6,0,2,20,3,12,12,0,0,1_1_1_2_22
6,0,0,0,6,0,1,20,9,6,12,1,0,1_1_1_7_9
7,0,0,0,6,0,1,20,6,10,12,0,0,1_1_1_7_11
8,0,0,0,6,0,1,19,9,8,11,1,0,1_1_1_7_15
9,0,0,0,7,0,1,19,3,14,11,0,0,1_1_1_7_20


In [79]:
cross_85_88_89 = cross(cross_85_88.to_dict("records"), df89.to_dict("records"), limits, threshold=0)
cross_85_88_89.head(10)

241663


,p80,p81,p82,p83,p84,p85,p86,p87,p88,p89,p90,p91,p92,p00
0,0,0,0,6,0,1,18,14,30,13,1,2,3,1_1_1_9_11_18_19_19
1,0,0,0,6,0,1,19,12,31,13,1,2,3,1_1_1_9_11_18_19_21
2,0,0,0,6,0,1,20,10,32,13,1,2,3,1_1_1_9_11_18_21_21
3,0,0,0,7,0,1,17,13,31,13,1,2,3,1_1_1_9_20_14_19_19
4,0,0,0,7,0,1,18,11,32,13,1,2,3,1_1_1_9_20_14_19_21
5,0,0,0,7,0,1,19,9,33,13,1,2,3,1_1_1_9_20_14_21_21
6,0,0,0,7,0,1,17,11,34,12,1,2,3,1_1_1_9_20_18_19_19
7,0,0,0,7,0,1,18,9,35,12,1,2,3,1_1_1_9_20_18_19_21
8,0,0,0,7,0,1,19,7,36,12,1,2,3,1_1_1_9_20_18_21_21
9,0,0,0,6,0,1,17,15,30,13,1,2,3,1_1_1_9_22_14_19_19


In [80]:
cross_85_88_89.shape, df87.shape

((154318, 14), (66770, 10))

In [82]:
def calculate_counter(row, limits):
    counter = 0
    
    for i in range(80, 92):
        if f"p{i}" not in row:
            continue
        
        if row[f"p{i}"] > limits[i]:
            counter += 1
            
    return counter

In [83]:
cross_85_88_89["counter"] = cross_85_88_89.apply(lambda x: calculate_counter(x, limits), axis=1)
df87["counter"] = df87.apply(lambda x: calculate_counter(x, limits), axis=1)

In [84]:
cross_85_88_89["counter"].value_counts()

0    154318
Name: counter, dtype: int64

In [85]:
df87["counter"].value_counts()

0    66770
Name: counter, dtype: int64

In [86]:
del df89, df88, df85, map85, map87, map88, map89, cross_85_88
gc.collect()

690

In [88]:
del cross_85_88_89["counter"], df87["counter"]

In [116]:
test1 = (
    df87
    .loc[df87["p90"].eq(1) & df87["p89"].eq(9) & df87["p83"].ge(4) & df87["p88"].le(6)]
)

test1

,p82,p83,p84,p85,p86,p87,p88,p89,p90,p00
36826,0,4,1,25,7,14,5,9,1,3_3_3_3_9_16
36874,0,4,1,24,7,16,4,9,1,3_3_3_3_15_16
37007,0,4,3,23,7,13,6,9,1,3_3_3_4_15_16
37350,0,4,1,21,13,12,5,9,1,3_3_3_9_12_16
37376,0,4,1,19,16,11,5,9,1,3_3_3_9_16_17
37475,0,4,1,20,13,14,4,9,1,3_3_3_12_15_16
37539,0,4,1,18,16,13,4,9,1,3_3_3_15_16_17
38163,0,4,3,19,13,11,6,9,1,3_3_4_12_15_16
38229,0,4,3,17,16,10,6,9,1,3_3_4_15_16_17
39496,0,4,1,17,19,10,5,9,1,3_3_9_12_12_16


In [117]:
test2 = (
    cross_85_88_89
    .loc[cross_85_88_89["p92"].eq(3) & cross_85_88_89["p91"].eq(2) & cross_85_88_89["p90"].le(2) & cross_85_88_89["p89"].eq(4) & cross_85_88_89["p88"].le(48) & cross_85_88_89["p83"].ge(6)]
    .sort_values(["p89", "p88", "p86"], ascending=[True, True, False])
)

test2

,p80,p81,p82,p83,p84,p85,p86,p87,p88,p89,p90,p91,p92,p00
6585,0,0,0,6,0,1,5,18,48,4,1,2,3,1_11_15_22_22_18_19_19
7479,0,0,0,6,0,1,4,19,48,4,1,2,3,1_15_22_22_22_14_19_19
21986,0,0,0,6,0,3,1,20,48,4,1,2,3,3_11_15_22_22_18_19_19
22878,0,0,0,6,0,3,0,21,48,4,1,2,3,3_15_22_22_22_14_19_19


In [120]:
crosses = cross(test1.to_dict("records"), test2.to_dict("records"), limits, threshold=0)

crosses.head()

74


,p80,p81,p82,p83,p84,p85,p86,p87,p88,p89,p90,p91,p92,p00
0,0,0,0,10,1,26,12,32,53,13,2,2,3,3_3_3_3_9_16_1_11_15_22_22_18_19_19
1,0,0,0,10,3,24,12,31,54,13,2,2,3,3_3_3_4_15_16_1_11_15_22_22_18_19_19
2,0,0,0,10,3,24,11,32,54,13,2,2,3,3_3_3_4_15_16_1_15_22_22_22_14_19_19
3,0,0,0,10,1,22,18,30,53,13,2,2,3,3_3_3_9_12_16_1_11_15_22_22_18_19_19
4,0,0,0,10,1,22,17,31,53,13,2,2,3,3_3_3_9_12_16_1_15_22_22_22_14_19_19


In [123]:
# 87
test1[test1["p00"].eq("3_12_12_15_16_17")]

,p82,p83,p84,p85,p86,p87,p88,p89,p90,p00
46187,0,4,1,10,28,9,4,9,1,3_12_12_15_16_17


In [135]:
comb[87][15]

{'p82': 0,
 'p83': 4,
 'p84': 1,
 'p85': 0,
 'p86': 1,
 'p87': 0,
 'p88': 4,
 'p89': 0,
 'p90': 1,
 'p00': 16}

In [124]:
# 85x1, 88x4, 89x3
test2[test2["p00"].eq("1_11_15_22_22_18_19_19")]

,p80,p81,p82,p83,p84,p85,p86,p87,p88,p89,p90,p91,p92,p00
6585,0,0,0,6,0,1,5,18,48,4,1,2,3,1_11_15_22_22_18_19_19


In [129]:
comb[89][17]

{'p84': 0,
 'p85': 0,
 'p86': 0,
 'p87': 1,
 'p88': 6,
 'p89': 3,
 'p90': 0,
 'p91': 0,
 'p92': 1,
 'p00': 18}

In [128]:
comb[89][18]

{'p84': 0,
 'p85': 0,
 'p86': 0,
 'p87': 2,
 'p88': 7,
 'p89': 0,
 'p90': 0,
 'p91': 1,
 'p92': 1,
 'p00': 19}

In [127]:
comb[85][0]

{'p80': 0,
 'p81': 0,
 'p82': 0,
 'p83': 6,
 'p84': 0,
 'p85': 1,
 'p86': 4,
 'p87': 0,
 'p88': 0,
 'p00': 1}

In [137]:
limits

{92: 4,
 91: 2,
 90: 2,
 89: 13,
 88: 54,
 87: 32,
 86: 34,
 85: 29,
 84: 44,
 83: 49,
 82: 99,
 81: 99,
 80: 99}

In [122]:
crosses.sort_values(["p88", "p87", "p86", "p85"])

,p80,p81,p82,p83,p84,p85,p86,p87,p88,p89,p90,p91,p92,p00
49,0,0,0,10,1,11,33,27,52,13,2,2,3,3_12_12_15_16_17_1_11_15_22_22_18_19_19
33,0,0,0,10,1,13,30,28,52,13,2,2,3,3_3_15_16_17_17_1_11_15_22_22_18_19_19
50,0,0,0,10,1,11,32,28,52,13,2,2,3,3_12_12_15_16_17_1_15_22_22_22_14_19_19
30,0,0,0,10,1,15,27,29,52,13,2,2,3,3_3_12_15_16_17_1_11_15_22_22_18_19_19
34,0,0,0,10,1,13,29,29,52,13,2,2,3,3_3_15_16_17_17_1_15_22_22_22_14_19_19
52,0,0,0,10,1,11,31,29,52,13,2,2,3,3_12_15_16_17_17_3_15_22_22_22_14_19_19
28,0,0,0,10,1,17,24,30,52,13,2,2,3,3_3_12_12_15_16_1_11_15_22_22_18_19_19
31,0,0,0,10,1,15,26,30,52,13,2,2,3,3_3_12_15_16_17_1_15_22_22_22_14_19_19
51,0,0,0,10,1,13,28,30,52,13,2,2,3,3_12_12_15_16_17_3_15_22_22_22_14_19_19
10,0,0,0,10,1,19,21,31,52,13,2,2,3,3_3_3_15_16_17_1_11_15_22_22_18_19_19


# ORTools

In [3]:
from ortools.sat.python import cp_model

In [7]:
from optimize import create_var
from main import preprocess_data_2

In [8]:
new_df = preprocess_data_2(df)

In [9]:
num_cnts = [new_df.shape[0], new_df.Club.nunique(), new_df.League.nunique(), new_df.Country.nunique()]

In [10]:
num_cnts

[1567, 320, 48, 76]

In [12]:
def get_dict(df, col):
    '''Map fields to a unique index'''
    d = {}
    unique_col = df[col].unique()
    for i, val in enumerate(unique_col):
        d[val] = i
    return d

In [13]:
map_idx= {} # Map fields to a unique index

fields = ["Club", "League", "Country", "Position", "Rating", "Color", "Rarity", "Name"]
for field in fields:
    map_idx[field] = get_dict(new_df, field)

In [22]:
expr = players_grouped["Country"].get(0, [])

In [15]:
model = cp_model.CpModel()

In [74]:
var_upper_bound = max(50, 45, 37)
x = model.NewIntVar(0, var_upper_bound, "x")
y = model.NewIntVar(0, var_upper_bound, "y")
z = model.NewIntVar(0, var_upper_bound, "z")

In [75]:
model.Add(2 * x + 7 * y + 3 * z <= 50)
model.Add(3 * x - 5 * y + 7 * z <= 45)
model.Add(5 * x + 2 * y - 6 * z <= 37)

In [79]:
model.Maximize(2 * x + 2 * y + 3 * z)

In [80]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [81]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"x = {solver.Value(x)}")
    print(f"y = {solver.Value(y)}")
    print(f"z = {solver.Value(z)}")
else:
    print("No solution found.")

x = 7
y = 3
z = 5


In [176]:
def create_map_idx(df):
    map_idx = {}
    
    for field in ["Team", "League", "Nation", "Rarity"]:
        curr_unique = df[field].unique().tolist()
        map_idx[field] = {}
        
        for idx, uniq in enumerate(curr_unique):
            map_idx[field][uniq] = idx
    
    return map_idx

In [177]:
map_idx = create_map_idx(processed_df)

In [178]:
model = cp_model.CpModel()

In [222]:
def create_variables(df, map_idx, model):
    num_players = df.shape[0]
    num_nations = len(map_idx["Nation"])
    num_leagues = len(map_idx["League"])
    num_teams = len(map_idx["Team"])
    
    players_group = {
        "Team": {}, "League": {}, "Nation": {}, "Rarity": {}
    }
    
    players = []
    for i in range(num_players):
        curr_name = f"{df.at[i, 'Name']}_{df.at[i, 'Nation']}_{df.at[i, 'League']}_{df.at[i, 'Team']}"
        players.append(model.NewBoolVar(curr_name))
    
    for i in range(num_players):
        curr_team_idx = map_idx["Team"][df.at[i, "Team"]]
        curr_league_idx = map_idx["League"][df.at[i, "League"]]
        curr_nation_idx = map_idx["Nation"][df.at[i, "Nation"]]
        curr_rarity_idx = map_idx["Rarity"][df.at[i, "Rarity"]]
                
        players_group["Team"][curr_team_idx] = players_group["Team"].get(curr_team_idx, []) + [players[i]]
        players_group["League"][curr_league_idx] = players_group["League"].get(curr_league_idx, []) + [players[i]]
        players_group["Nation"][curr_nation_idx] = players_group["Nation"].get(curr_nation_idx, []) + [players[i]]
        players_group["Rarity"][curr_rarity_idx] = players_group["Rarity"].get(curr_rarity_idx, []) + [players[i]]
    
    leagues = [model.NewBoolVar(f"league_{i}") for i in range(num_leagues)]
    nations = [model.NewBoolVar(f"nation_{i}") for i in range(num_nations)]
    teams = [model.NewBoolVar(f"team_{i}") for i in range(num_teams)]
    
    return players, players_group, leagues, nations, teams

In [223]:
players, players_group, leagues, nations, teams = create_variables(processed_df, map_idx, model)

In [210]:
def create_league_constraint(leagues, players, players_group, model, bound):
    num_leagues = len(players_group["League"])
    
    for i in range(num_leagues):
        exp = players_group["League"][i]
        model.Add(cp_model.LinearExpr.Sum(exp) >= 1).OnlyEnforceIf(leagues[i])
        model.Add(cp_model.LinearExpr.Sum(exp) == 0).OnlyEnforceIf(leagues[i].Not())
        
    model.Add(cp_model.LinearExpr.Sum(leagues) <= bound)
    
    return model

In [211]:
def create_team_constraint(players, players_group, model, bound):
    num_teams = len(players_group["Team"])
    
    for i in range(num_teams):
        exp = players_group["Team"][i]
        model.Add(cp_model.LinearExpr.Sum(exp) <= bound)
    
    return model

In [212]:
def create_nation_constraint(nations, players, players_group, model, bound):
    num_nations = len(players_group["Nation"])
    
    for i in range(num_nations):
        exp = players_group["Nation"][i]
        model.Add(cp_model.LinearExpr.Sum(exp) >= bound).OnlyEnforceIf(nations[i])
        model.Add(cp_model.LinearExpr.Sum(exp) < bound).OnlyEnforceIf(nations[i].Not())
    
    model.AddAtLeastOne(nations)
    
    return model

In [213]:
def create_rarity_constraint(players_group, model, bound, map_idx, rarity):
    idx = map_idx["Rarity"][rarity]
    exp = players_group["Rarity"][idx]
    
    model.Add(cp_model.LinearExpr.Sum(exp) >= bound)
    
    return model

In [214]:
def create_no_of_players_constraint(players, model, bound):
    model.Add(cp_model.LinearExpr.Sum(players) == bound)
    
    return model

In [215]:
def set_objective(df, players, model):
    costs = df["ExternalPrice"].to_list()
    model.Minimize(cp_model.LinearExpr.WeightedSum(players, costs))
    
    return model

In [224]:
model = create_league_constraint(leagues, players, players_group, model, 4)
model = create_team_constraint(players, players_group, model, 3)
model = create_nation_constraint(nations, players, players_group, model, 4)
model = create_rarity_constraint(players_group, model, 9, map_idx, "Rare")
model = create_no_of_players_constraint(players, model, 11)

model = set_objective(processed_df, players, model)

In [225]:
solver = cp_model.CpSolver()
# solver.parameters.log_search_progress = True

In [226]:
solver.Solve(model)

4

In [227]:
df_indices = []

for i in range(processed_df.shape[0]):
    if solver.Value(players[i]) == 1:
        df_indices.append(i)
        
processed_df.loc[df_indices, :]

,Name,Rarity,Nation,League,Team,ExternalPrice
454,Portanova,Common,Italy,Serie BKT,Reggiana,0
534,McDonald,Rare,Scotland,EFL League Two,Bradford City,200
538,Çelik,Rare,Turkey,Trendyol Süper Lig,Samsunspor,200
544,Türkmen,Rare,Turkey,Trendyol Süper Lig,MKE Ankaragücü,250
545,Topçu,Rare,Turkey,Trendyol Süper Lig,Çaykur Rizespor,250
549,Tarnat,Rare,Germany,3. Liga,1860 München,200
557,Khan,Rare,Pakistan,EFL League Two,Grimsby Town,200
584,Sauter,Rare,Germany,3. Liga,SC Freiburg II,200
590,Michel,Rare,Germany,3. Liga,B. Dortmund II,200
591,Özkan,Rare,Turkey,Trendyol Süper Lig,Antalyaspor,250


In [122]:
model = cp_model.CpModel()

In [123]:
tests = [model.NewBoolVar(f"test_{i}") for i in range(100)]

In [124]:
costs = [i for i in range(100)]

In [125]:
model.Add(cp_model.LinearExpr.Sum(tests) == 10)

In [126]:
model.Minimize(cp_model.LinearExpr.WeightedSum(tests, costs))

In [127]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [4]:
processed_df = df.loc[df["Rarity"].isin(["Rare", "Common"]), ["Name", "Rarity", "Nation", "League", "Team", "ExternalPrice"]].reset_index(drop=True)

In [5]:
processed_df.drop_duplicates(ignore_index=True, inplace=True)

In [6]:
processed_df

,Name,Rarity,Nation,League,Team,ExternalPrice
0,Haaland,Rare,Norway,Premier League,Manchester City,70000
1,Kerr,Rare,Australia,Barclays WSL,Chelsea,55000
2,Courtois,Rare,Belgium,LALIGA EA SPORTS,Real Madrid,50000
3,Benzema,Rare,France,ROSHN Saudi League,Al Ittihad,50000
4,van Dijk,Rare,Netherlands,Premier League,Liverpool,122000
...,...,...,...,...,...,...
615,Mu,Common,China PR,CSL,Qingdao Hainiu,200
616,Woltman,Common,England,EFL League One,Oxford United,2500
617,Hofmann,Common,Germany,Bundesliga 2,1. FC Nürnberg,200
618,Smith,Common,Wales,EFL League One,Wigan Athletic,200
